In [67]:
'''
The code is used to train BC imitator, or pretrained GAIL imitator
'''

import argparse
import tempfile
import os.path as osp
import gym
import logging
from tqdm import tqdm

import tensorflow as tf

from baselines.gail import mlp_policy
from baselines import bench
from baselines import logger
from baselines.common import set_global_seeds, tf_util as U
from baselines.common.misc_util import boolean_flag

#from baselines.gail.run_mujoco import runner
from baselines.gail.dataset.mujoco_dset import Mujoco_Dset 

##from baselines.common.mpi_adam import MpiAdam

In [5]:
from gfrl.base.run_my_ppo2 import create_single_scenic_environment
network = "gfootball_impala_cnn"
scenario = "../_scenarios/academy/pass_n_shoot.scenic"
load_path = "/home/ubuntu/ScenicGFootBall/training/gfrl/_saved_models/pass_shoot_5M"



In [6]:


import tensorflow.compat.v1 as tf
import multiprocessing

from baselines.common.vec_env.dummy_vec_env import DummyVecEnv
vec_env = DummyVecEnv([lambda _i=i: \
                        create_single_scenic_environment(_i,scenario) for i in
                        range(1)])



ncpu = multiprocessing.cpu_count()
config = tf.ConfigProto(allow_soft_placement=True,
                        intra_op_parallelism_threads=ncpu,
                        inter_op_parallelism_threads=ncpu)
config.gpu_options.allow_growth = True
tf.Session(config=config).__enter__()

from gfrl.common.mybase import ppo2
model = ppo2.learn(
    network=network,
    total_timesteps=0,
    env=vec_env,
    nsteps=512,
    load_path=load_path
    )




/home/ubuntu/.local/lib/python3.7/site-packages/scenic/core/errors.py:160: UserWarning: unable to install sys.excepthook to format Scenic backtraces
  warnings.warn('unable to install sys.excepthook to format Scenic backtraces')


Scenic Environment:  ../_scenarios/academy/pass_n_shoot.scenic
pygame 2.0.1 (SDL 2.0.14, Python 3.7.10)
Hello from the pygame community. https://www.pygame.org/contribute.html





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






In [55]:
from gfrl.dm.my_runner import MyTrajRunner

from baselines.common.vec_env.dummy_vec_env import DummyVecEnv
vec_env = DummyVecEnv([lambda _i=i: \
                        create_single_scenic_environment(_i,scenario) for i in
                        range(1)])

runner = MyTrajRunner(env=vec_env, model=model, nsteps=512, num_episodes=200)
mb_obs, mb_rewards, mb_dones, mb_actions, mb_values, mb_neglogpacs, mb_states, epinfos, mb_gt = runner.run()



Scenic Environment:  ../_scenarios/academy/pass_n_shoot.scenic
in run


In [64]:
import numpy as np
np.mean(mb_rewards)

0.031296574

In [65]:
print(mb_obs.shape, mb_actions.shape)

(6039, 72, 96, 16) (6039,)


In [66]:
mb_actions.reshape(-1,1).shape

(6039, 1)

In [58]:
mb_act_oh = np.zeros((mb_actions.shape[0], mb_actions.max()+1))
mb_act_oh[np.arange(mb_actions.shape[0]), mb_actions] = 1

In [59]:
mb_act_oh.shape

(6039, 19)

In [60]:
mb_actions[0:5]

array([ 9,  9,  9,  9, 13])

In [61]:
mb_act_oh[0:5]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0.]])

In [62]:
expert_file_name = "../_data/pns_200.npz"
import numpy as np 
np.savez(expert_file_name, obs=mb_obs, acs=mb_act_oh)

In [10]:

#expert_file_name = "../_data/pns_100.npz"
#import numpy as np 
#np.savez(expert_file_name, obs=mb_obs, acs=mb_actions)


In [63]:
td = np.load(expert_file_name)

In [12]:
list(td.keys())

['obs', 'acs']

In [13]:
class Dset(object):
    def __init__(self, inputs, labels, randomize):
        self.inputs = inputs
        self.labels = labels
        assert len(self.inputs) == len(self.labels)
        self.randomize = randomize
        self.num_pairs = len(inputs)
        self.init_pointer()

    def init_pointer(self):
        self.pointer = 0
        if self.randomize:
            idx = np.arange(self.num_pairs)
            np.random.shuffle(idx)
            self.inputs = self.inputs[idx, :]
            self.labels = self.labels[idx, :]

    def get_next_batch(self, batch_size):
        # if batch_size is negative -> return all
        if batch_size < 0:
            return self.inputs, self.labels
        if self.pointer + batch_size >= self.num_pairs:
            self.init_pointer()
        end = self.pointer + batch_size
        inputs = self.inputs[self.pointer:end, :]
        labels = self.labels[self.pointer:end, :]
        self.pointer = end
        return inputs, labels

In [14]:
class GFDset(object):
    def __init__(self, expert_path):
        traj_data = np.load(expert_path)
        self.inputs = traj_data["obs"]
        self.labels = labels["acs"]
        assert len(self.inputs) == len(self.labels)
        self.randomize = randomize
        self.num_pairs = len(inputs)
        self.init_pointer()

    def init_pointer(self):
        self.pointer = 0
        if self.randomize:
            idx = np.arange(self.num_pairs)
            np.random.shuffle(idx)
            self.inputs = self.inputs[idx, :]
            self.labels = self.labels[idx, :]

    def get_next_batch(self, batch_size):
        # if batch_size is negative -> return all
        if batch_size < 0:
            return self.inputs, self.labels
        if self.pointer + batch_size >= self.num_pairs:
            self.init_pointer()
        end = self.pointer + batch_size
        inputs = self.inputs[self.pointer:end, :]
        labels = self.labels[self.pointer:end, :]
        self.pointer = end
        return inputs, labels

In [15]:
class MujocoEditDset(Dset):
    
    def __init__(self, expert_path, train_fraction=0.7, traj_limitation=-1, randomize=True):
        traj_data = np.load(expert_path)
        if traj_limitation < 0:
            traj_limitation = len(traj_data['obs'])
        obs = traj_data['obs'][:traj_limitation]
        acs = traj_data['acs'][:traj_limitation]

        """
        # obs, acs: shape (N, L, ) + S where N = # episodes, L = episode length
        # and S is the environment observation/action space.
        # Flatten to (N * L, prod(S))
        if len(obs.shape) > 2:
            self.obs = np.reshape(obs, [-1, np.prod(obs.shape[2:])])
            self.acs = np.reshape(acs, [-1, np.prod(acs.shape[2:])])
        else:
            self.obs = np.vstack(obs)
            self.acs = np.vstack(acs)
        """

        self.rets = traj_data['ep_rets'][:traj_limitation]
        self.avg_ret = sum(self.rets)/len(self.rets)
        self.std_ret = np.std(np.array(self.rets))
        if len(self.acs) > 2:
            self.acs = np.squeeze(self.acs)
        assert len(self.obs) == len(self.acs)
        self.num_traj = min(traj_limitation, len(traj_data['obs']))
        self.num_transition = len(self.obs)
        self.randomize = randomize
        self.dset = Dset(self.obs, self.acs, self.randomize)
        # for behavior cloning
        self.train_set = Dset(self.obs[:int(self.num_transition*train_fraction), :],
                              self.acs[:int(self.num_transition*train_fraction), :],
                              self.randomize)
        self.val_set = Dset(self.obs[int(self.num_transition*train_fraction):, :],
                            self.acs[int(self.num_transition*train_fraction):, :],
                            self.randomize)
        self.log_info()

    def log_info(self):
        logger.log("Total trajectories: %d" % self.num_traj)
        logger.log("Total transitions: %d" % self.num_transition)
        logger.log("Average returns: %f" % self.avg_ret)
        logger.log("Std for returns: %f" % self.std_ret)

    def get_next_batch(self, batch_size, split=None):
        if split is None:
            return self.dset.get_next_batch(batch_size)
        elif split == 'train':
            return self.train_set.get_next_batch(batch_size)
        elif split == 'val':
            return self.val_set.get_next_batch(batch_size)
        else:
            raise NotImplementedError

    def plot(self):
        import matplotlib.pyplot as plt
        plt.hist(self.rets)
        plt.savefig("histogram_rets.png")
        plt.close()
    

In [16]:
from gfrl.base.run_my_ppo2 import create_single_scenic_environment
    
scenario = "../_scenarios/academy/pass_n_shoot.scenic"
env = create_single_scenic_environment(0,scenario)

from baselines.common.policies import build_policy
network = "gfootball_impala_cnn"
#Check Network parameters
network_kwargs = {}
policy_fn = build_policy(env, network, **network_kwargs)

Scenic Environment:  ../_scenarios/academy/pass_n_shoot.scenic


In [17]:

policy_with_value = policy_fn()

In [18]:
policy_with_value

['X',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_evaluate',
 'action',
 'initial_state',
 'load',
 'neglogp',
 'pd',
 'pdtype',
 'pi',
 'save',
 'sess',
 'state',
 'step',
 'value',
 'vf']